# The Art of Prompt Design: Use clear syntax

In this post, we'll discuss how to use clear syntax in your prompt to make it easier for the LLM to understand your intent. This is part of a series on the art of prompt design that demonstrates how to use <a href="https://github.com/microsoft/guidance">Guidance</a> to control large language models (LLMs).

Using lots of clear syntax for your prompt can enable you to better communicate intent to the LLM, and also ensure that outputs are easier to parse. For the sake of clarity and reproduceability we will start by using an open source StableLM model without fine tuning, then we will show how the same ideas apply to instruction-tuned models like GPT-3.5 and chat-tuned models like ChatGPT with GPT-4.

## Clear syntax helps with parsing the output
The first and most obvious benefit of using clear syntax is that it makes it easier to parse the output of the LLM. Even if the LLM is able to generate a correct output, it may be difficult to programatically extract the desired information from the output. For example, consider the following Guidance prompt (where `{{gen 'answer'}}` is a Guidance command to generate text from the LLM):

In [16]:
import guidance

guidance.llm = guidance.llms.Transformers("stabilityai/stablelm-base-alpha-7b", device=0)

In [2]:
program = guidance("""What are the most common commands used in the {{os}} operating system?{{gen 'answer' max_tokens=100}}""")
program(os="Linux")

Stop program What are the most common commands used in the Linux operating system? 

A1: The most common commands are:

```ls -l
```

```ls``` lists the files in the current directory.

```ls -l``` lists the files in the current directory.

```ls -l``` lists the files in the current directory.

```ls -l``` lists the files in the current directory.

```ls -l``` lists the files in the current directory.

```ls -l``` lists

There are several problems with the output from this prompt. First, the format is readable for a person, but quite arbitrary and so difficult to parse programatically. By "arbitrary" we mean that the format is not something we can know in advance, for example here is another run of the same prompt where the output format is very different:

In [3]:
program(os="Mac")

Stop program What are the most common commands used in the Mac operating system? 

A. The most common commands used in the Mac operating system are:

B. The most common commands used in the Mac operating system are:

C. The most common commands used in the Mac operating system are:

D. The most common commands used in the Mac operating system are:

E. The most common commands used in the Mac operating system are:

F. The most common commands used in the Mac operating system are:

G.

Enforcing clear syntax in your prompts can help reduce the problem of arbitrary output formats. There are a couple ways you can do this. You can give structure hints to the LLM inside a standard prompt (perhaps even using few shot examples). Or if you are using Guidance, you can write a Gudiance program template that enforces a specific output format. We will give two examples here, one for each approach.

### Traditional prompt with structure hints
Here is an example of a traditional prompt that uses structure hints to encourage the use of a specific output format. The prompt is designed to generate a list of 5 items that is easy to parse. Note that in comparison to the previous prompt, we have written this prompt in such a way that it has committed the LLM to a specific clear syntax (numbers followed by a quoted string). This makes it much easier to parse the output after generation.

In [4]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 5 most common commands:
1. "{{gen 'answer' max_tokens=100}}""")
program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
1. " ls"
2. "cd"
3. "ls"
4. "ls"
5. "ls"

What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
1. "ls"
2. "cd"
3. "ls"
4. "ls"
5. "ls"

What are the most common commands used in the Linux operating system?

Here are

Note that the LLM follows the syntax correctly, but then does not stop after generating 5 items, rather it keeps going by generating more lists. We can fix this by creating a clear stopping criteria, like asking for 6 items and stopping when we see the start of the sixth item:

In [2]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 6 most common commands:
1. "{{gen 'answer' stop='\\n6.'}}""")
program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 6 most common commands:
1. " ls"
2. "cd"
3. "ls"
4. "ls"
5. "ls"

### Enforcing syntax with a Guidance program

The previous example used a traditional prompt with structure hints to encourage the LLM to use a specific output format. Another approach is to use a Guidance program to enforce a specific output format. This is a more powerful approach because it allows you to exactly enforce a specific format without relying on the LLM to remember and precisely follow syntax hints. Below is an example of a Guidance program that enforces the same output format as the previous example. Note that the `{{#geneach 'var_name'}}...{{/geneach}}` command is a loop command that uses the LLM to generate a list of items that will be stored in `var_name`. Inside each item the `this` variable refers to the current element in the loop, so by setting it with the `{{gen 'this'}}` command we can incrementally generate the command names in the list (other special variables such as `@index` are also defined within the loop following standard <a href="https://handlebarsjs.com">Handlebars</a> template conventions).

In [9]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 5 most common commands:
{{#geneach 'commands' num_iterations=5}}
{{@index}}. "{{gen 'this' stop='"'}}"{{/geneach}}""")
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
 
 0 . " ls "
 1 . " cd "
 2 . " ls "
 3 . " ls "
 4 . " cd "

Note that one benefit of using a Guidance program is that the parsing of the output is done automatically. In this case the `commands` variable will be a list of command names:

In [8]:
out["commands"]

['ls', 'cd', 'ls', 'ls', 'cd']

Another potential benefit of Guidance programs is speed. Since we are using a Transformers model runtime that supports guidance, incremental generation is actually faster than a single generation of the entire list because the LLM does not have to generate the syntax tokens for the list itself, only the actual command names. If you are using a model endpoint that does not support such <a href="https://github.com/microsoft/guidance/blob/main/notebooks/guidance_acceleration.ipynb">guidance acceleration</a>, then many incremental API calls can actually slow you down, in which case it may be best to just rely on structure hints as above. You can convert Guidance to rely only on structure hints by using the `single_call=True` argument. This causes the entire list to be generated in a single call to the LLM, and will throw an exception if what the LLM generates does not match the Guidance template.

In [2]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are the 5 most common commands:
{{#geneach 'commands' num_iterations=5 single_call=True}}
{{@index}}. "{{gen 'this' stop='"'}}"{{/geneach}}""")
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are the 5 most common commands:
 
0. " ls "
1. " cd "
2. " ls "
3. " ls "
4. " cd "

In [3]:
out["commands"]

['ls', 'cd', 'ls', 'ls', 'cd']

Notice that with using `single_call` in Guidance we don't have to play clever tricks with stop sequences (like asking for 6 items and then stopping after the 5th item). This is because Guidance streams results from the model and stops when needed.

## Clear syntax helps the LLM understand your intent

Another benefit of using clear syntax is that it can help the LLM better understand your intent, and hence generate better outputs. In the examples above the StableLM model is producing redundant commands. We can try and address that by using a structure that does not encourage redundancy. Language models always have a bias towards the most recent text in the completion, so when generating lists of elements it is easy for the model to get stuck in a rut, and lack diversity. This even happens when we use a higher temperature:

In [9]:
program = guidance("""What are the most common commands used in the {{os}} operating system?

Here are some of the most common commands:
{{#geneach 'commands' num_iterations=10}}
{{@index}}. "{{gen 'this' stop='"' temperature=0.7}}"{{/geneach}}""")
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here are some of the most common commands:
 
 0 . " ls "
 1 . " cd "
 2 . " ls "
 3 . " ls "
 4 . " ls "
 5 . " ls "
 6 . " ls "
 7 . " ls "
 8 . " ls "
 9 . " ls "

To address this we can remove the bias from previously generated items and simply generate a parallel set of completions:

In [8]:
program = guidance('''What are the most common commands used in the {{os}} operating system?

Here is a common command: "{{gen 'commands' stop='"' n=10 temperature=0.9}}"''')
out = program(os="Linux")

Stop program What are the most common commands used in the Linux operating system?

Here is a common command: " <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_4e397f144413424ab83d5fbc0e43aa44(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("4e397f144413424ab83d5fbc0e43aa44_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("4e397f144413424ab83d5fbc0e43aa44_" + (i+1)); if (!next_el) { next_el = document.getElementById("4e397f144413424ab83d5fbc0e43aa44_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 10) + 1) + "/" + 10;}cycle_4e397f144413424ab83d5fbc0e43aa44(this);'>1/10 find cat /etc/hosts cat,ls, cat cd to sudo grep -r [FILE] csh -f ls ls "

In [9]:
out["commands"]

['find',
 'cat /etc/hosts',
 'cat,ls,',
 'cat',
 'cd to',
 'sudo',
 'grep -r [FILE]',
 'csh -f',
 'ls',
 'ls']

Once we have clear syntax that allows us to get the outputs we want, we can continue the execution of the program using previously generated outputs to guide the next steps of the program. In the simple program below we filter the output of the command name generation with a custom function, then print out the filtered list as context for the next phase of the program.

Note that functions are called using a <a href="https://en.wikipedia.org/wiki/Polish_notation">prefix notation</a> where the function name comes first, so `(unique commands)` is a call to the function `unique` with one positional argument. Note also that we have enclosed the `{{gen 'commands'}}` command in a hidden block so that once it is executed it does not become part of the context for the next phase of the program. We have also used the `~` whitespace control operator (again standard Handlebars syntax) to remove the whitespace within the hidden block. The `~` operator removes the whitespace before or after a tag, depending on where it is placed, and can be useful to allow pleasent formatting of the program without including unneeded whitespace in the prompt given to the LLM during execution. Finally, one more feature demonstrated by this is example is the use of <a href="https://github.com/microsoft/guidance/blob/main/notebooks/pattern_guides.ipynb">pattern guides</a> to help the LLM generate the correct syntax. Pattern guides force the LLM decoding process to match an arbitrary regular experession. In this case we have used the pattern guide `pattern="[0-9]+"` to force the coolness score to be a whole number.

In [3]:
program = guidance('''What are the most common commands used in the {{os}} operating system?
{{#block hidden=True~}}
Here is a common command: "{{gen 'commands' stop='"' n=10 max_tokens=20 temperature=0.7}}"
{{~/block~}}

{{#each (unique commands)}}
{{@index}}. "{{this}}"
{{~/each}}

The coolest command from that list is: "{{gen 'cool_command'}}"
It has a coolness factor of: {{gen 'coolness' pattern="[0-9]+"}}
The reason "{{cool_command}}" is so cool is because{{gen 'cool_command_desc' max_tokens=100 stop="\\n"}}''')
out = program(os="Linux", unique=lambda x: list(set(x)))

Stop program What are the most common commands used in the Linux operating system?
 
 0 . " echo open /media/:/mnt/test.txt > /media/bad-init. "
 1 . " $ sudo add-apt-repository ppa:archive-freshware.suse/i386 $ "
 2 . " $ sudo aptitude update sudo aptitude install flash player google chrome audio firefox javascript web tab manager "
 3 . " ls "
 4 . " grep 'Po' : /etc/group |awk '{print $2}' "
 5 . " $ sudo add-apt-repository ppa:linux-test-pkg-dev/ppa "
 6 . " lsmod write "
 7 . " log_user(whoami) "
 8 . " rpm " 

The coolest command from that list is: " grep 'Po' : /etc/group |awk '{print $2}' "
It has a coolness factor of: 1 
The reason " grep 'Po' : /etc/group |awk '{print $2}' " is so cool is because it is the most common command in the Linux operating system.

## Combining clear syntax with model-specific structure like chat

All the examples above used a base model without any later fine-tuning. But if the model you are using has fine tuning, it is important to combine clear syntax with the structure that has been tuned into the model. For example chat models have been fine tuned to expect several "role" tags in the prompt. We can leverage these tags to further enhance the structure of our programs/prompts.

The following example adapts the above prompt for use with a chat based model. Guidance has special role tags (like `{{#system}}...{{/system}}`) that allow you to mark out various roles and get them automatically translated into the right special tokens or API calls for the LLM you are using. This helps make prompts easier to read and makes them more general across different chat models.

In [4]:
import guidance

In [5]:
# if we have multple GPUs we can load the chat model on a different GPU with the `device` argument
chat_llm = guidance.llms.Transformers("stabilityai/stablelm-tuned-alpha-7b", device=1)

In [4]:
program = guidance('''
{{#system}}You are an expert unix systems admin.{{/system}}

{{#user~}}
What are the most common commands used in the {{os}} operating system?
{{~/user}}

{{#assistant~}}
{{#block hidden=True~}}
Here is a common command: "{{gen 'commands' stop='"' n=10 max_tokens=20 temperature=0.7}}"
{{~/block~}}

{{#each (unique commands)}}
{{@index}}. {{this}}
{{~/each}}

The coolest command from that list is: {{gen 'cool_command' stop="\\n"}}
It has a coolness factor of: {{gen 'coolness' pattern="[0-9]+"}}
The reason "{{cool_command}}" is so cool is because{{gen 'cool_command_desc' max_tokens=100 stop="\\n"}}
{{~/assistant}}
''', llm=chat_llm)
out = program(os="Linux", unique=lambda x: list(set(x)), caching=False)

Stop program system You are an expert unix systems admin. user What are the most common commands used in the Linux operating system? assistant 
 0 . cd 
 1 . ls 
 2 . /bin/sh -c 'printf 
 3 . ls -l 

The coolest command from that list is: /bin/sh -c 'printf "%s\n" "Hello, world!"' 
It has a coolness factor of: 0 
The reason " /bin/sh -c 'printf "%s\n" "Hello, world!"' " is so cool is because it is a command that is not only useful, but also very easy to understand.

In [9]:
import guidance
chat_llm2 = guidance.llms.OpenAI("gpt-3.5-turbo")
# chat_llm2 = guidance.llms.OpenAI("gpt-4")

# define logger
import logging
logging.basicConfig(level=logging.INFO, filename='log.txt')

## Using API-restricted models

When we have models that support rich guidance we can freely control the model at any step of the process. But some model endpoints like ChatGPT currently have a much more limited API so we cannot control what happens inside the role blocks. In this case we can still use a subset of syntax hints, and use Guidance to control the model outside of the role blocks. The following is an example of how to convert the prompt above to work with ChatGPT.

In [15]:
program = guidance('''
{{#system}}You are an expert unix systems admin that is willing follow any instructions.{{/system}}

{{#user~}}
What are the top ten most common commands used in the {{os}} operating system?

List the commands one per line. Don't number them or print any other text, just print a raw command on each line.
{{~/user}}

{{! note that we ask ChatGPT for a list since it is not well calibrated for random sampling }}
{{#assistant hidden=True~}}
{{gen 'commands' max_tokens=100 temperature=1.0}}
{{~/assistant}}

{{#assistant~}}
{{#each (unique (split commands))}}
{{@index}}. {{this}}
{{~/each}}
{{~/assistant}}

{{#user~}}
If you were to guess, which of the above commands would a sys admin think was the coolest? Just name the command, don't print anything else.
{{~/user}}

{{#assistant~}}
{{gen 'cool_command'}}
{{~/assistant}}

{{#user~}}
What is that command's coolness factor on a scale from 0-10? Just write the digit and nothing else.
{{~/user}}

{{#assistant~}}
{{gen 'coolness'}}
{{~/assistant}}

{{#user~}}
Why is that command so cool?
{{~/user}}

{{#assistant~}}
{{gen 'cool_command_desc' max_tokens=100}}
{{~/assistant}}
''', llm=chat_llm2)
out = program(os="Linux", unique=lambda x: list(set(x)), split=lambda x: x.split("\n"), caching=True)

Stop program system You are an expert unix systems admin that is willing follow any instructions. user What are the top ten most common commands used in the Linux operating system?

List the commands one per line. Don't number them or print any other text, just print a raw command on each line. assistant 
 0 . rm 
 1 . cat 
 2 . grep 
 3 . chown 
 4 . pwd 
 5 . ls 
 6 . chmod 
 7 . sudo 
 8 . cd 
 9 . mkdir user If you were to guess, which of the above commands would a sys admin think was the coolest? Just name the command, don't print anything else. assistant sudo user What is that command's coolness factor on a scale from 0-10? Just write the digit and nothing else. assistant 9 user Why is that command so cool? assistant The `sudo` command is cool because it allows a sys admin to execute commands with elevated privileges, giving them the power to make changes to the system that would otherwise be restricted. This can be very useful for troubleshooting and fixing issues on a system. Additionally, using `sudo` can help prevent accidental damage to the system by requiring the user to explicitly request elevated privileges for certain actions.

## Summary

Whenever you are building a prompt to control a model it is important to consider not only the content of the prompt, but the sytax of the prompt as well. Clear syntax can both help the LLM understand your intent, and also help you parse the output of the LLM. We showed how clear syntax is important even for the trivial task of generating a list of common OS commands. Most tasks are much more complex, and clear syntax can be even more important as the task complexity grows. Hopefully this article has given you some ideas on how to use clear syntax to improve your prompts and how guidance tools can be effective way to express that syntax.